In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report, precision_recall_curve, roc_curve

In [5]:
# Threshold
threshold = 34000

# Load the datasets
attack_label = pd.read_csv('/local/scratch/exported/MP_Defi_txs_TY_23/guanda/attack_label.csv')
unlabeled = pd.read_csv('/local/scratch/exported/MP_Defi_txs_TY_23/guanda/unlabeled.csv')

# Select features
features = ['from_address_profit', 'to_address_profit', 'highest_profit_in_usd',
            'highest_price_change_ratio', 'path_length', 'num_swap_events', 'flashloan_in_usd']

# Preprocessing
unlabeled['flashloan_in_usd'] = pd.to_numeric(unlabeled['flashloan_in_usd'], errors='coerce')
normal_label = unlabeled[unlabeled['highest_profit_in_usd'] <= 1000]
unlabeled = unlabeled[unlabeled['highest_profit_in_usd'] > 1000].reset_index(drop=True)

# Add a new column for labels: 1 for attack and 0 for normal
attack_label['label'] = 1  # Attack
normal_label['label'] = 0  # Normal

# Combine the datasets
combined_df = pd.concat([attack_label, normal_label], ignore_index=True)

# Convert 'flashloan_in_usd' to numeric, coercing errors to NaN
combined_df['flashloan_in_usd'] = pd.to_numeric(combined_df['flashloan_in_usd'], errors='coerce')

X = combined_df[features]
y = combined_df['label']
X_unlabeled = unlabeled[features]

# Combine X and X_unlabeled temporarily for imputation and scaling
X_combined = np.vstack([X, X_unlabeled])

# Handle missing values by imputing with the mean strategy
imputer = SimpleImputer(strategy='mean')
X_combined_imputed = imputer.fit_transform(X_combined)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined_imputed)

# Separate them back into X and X_unlabeled
X_scaled = X_combined_scaled[:len(X), :]
X_unlabeled_scaled = X_combined_scaled[len(X):, :]

temp = [
"0x3b19e152943f31fe0830b67315ddc89be9a066dc89174256e17bc8c2d35b5af8",
"0xcb0ad9da33ecabf75df0a24aabf8a4517e4a7c5b1b2f11fee3b6a1ad9299a282",
"0xcb58fb952914896b35d909136b9f719b71fc8bc60b59853459fc2476d4369c3a",
"0xf72f1d10fc6923f87279ce6c0aef46e372c6652a696f280b0465a301a92f2e26",
"0x118b7b7c11f9e9bd630ea84ef267b183b34021b667f4a3061f048207d266437a",
"0x3503253131644dd9f52802d071de74e456570374d586ddd640159cf6fb9b8ad8",
"0x35f8d2f572fceaac9288e5d462117850ef2694786992a8c3f6d02612277b0877",
"0x0fc6d2ca064fc841bc9b1c1fad1fbb97bcea5c9a1b2b66ef837f1227e06519a6",
"0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c",
"0x46a03488247425f845e444b9c10b52ba3c14927c687d38287c0faddc7471150a",
"0x8bb8dc5c7c830bac85fa48acad2505e9300a91c3ff239c9517d0cae33b595090",
"0xf6022012b73770e7e2177129e648980a82aab555f9ac88b8a9cda3ec44b30779",
"0xcd314668aaa9bbfebaf1a0bd2b6553d01dd58899c508d4729fa7311dc5d33ad7"
]

indices_temp = combined_df[combined_df['tx_hash'].isin(temp)].index

X_temp_scaled = X_scaled[indices_temp]
y_temp = y.iloc[indices_temp]

X_scaled_removed = np.delete(X_scaled, indices_temp, axis=0)
y_array = np.array(y) 
y_removed = np.delete(y_array, indices_temp, axis=0)

X_train, X_test, y_train, y_test = train_test_split(X_scaled_removed, y_removed, test_size=0.2, random_state=42, stratify=y_removed)

X_train = np.vstack([X_train, X_temp_scaled])
y_train = np.concatenate([y_train, y_temp])

unlabeled_predictions = pd.DataFrame()

# Random Forest Classifier

In [3]:
# Define the objective function for Optuna
def rf_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                   min_samples_split=min_samples_split, random_state=42)
    
    # Use StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=skf, n_jobs=-1, scoring='f1').mean()
    return score

# Check if the model already exists
if os.path.exists(f'Models/{threshold}/random_forest_model_{threshold}.pkl'):
    # Load the existing model
    best_model = joblib.load(f'Models/{threshold}/random_forest_model_{threshold}.pkl')
    print("Loaded existing Random Forest model.")
    best_params = best_model.get_params()
    for param, value in best_params.items():
        print(f"{param}: {value}")
else: 
    # Optimize with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(rf_objective, n_trials=50)

    # Best parameters
    best_params = study.best_params
    best_model = RandomForestClassifier(**best_params, random_state=42)
    # Save the model
    joblib.dump(best_model, f'Models/{threshold}/random_forest_model_{threshold}.pkl')
    print("Trained and saved new Random Forest model.")

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate confusion matrix and AUC-ROC score
conf_matrix = confusion_matrix(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

# Calculate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)

# Calculate precision, recall, and thresholds
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Create a DataFrame to store Precision-Recall data
pr_data_df = pd.DataFrame({
    'Recall': recall,
    'Precision': precision
})

# Save the Precision-Recall data to a CSV file
pr_data_df.to_csv('Models/random_forest_precision_recall_data.csv', index=False)

# Prepare confusion matrix data
tn, fp, fn, tp = conf_matrix.ravel()

# Create a DataFrame to store the metrics
metrics_list = [
    {'Metric': 'True Negatives', 'Value': tn},
    {'Metric': 'False Positives', 'Value': fp},
    {'Metric': 'False Negatives', 'Value': fn},
    {'Metric': 'True Positives', 'Value': tp},
    {'Metric': 'AUC-ROC', 'Value': auc_roc}
]

# Add classification report data to the metrics DataFrame
for label, metrics in class_report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        metrics_list.append({'Metric': f'Precision ({label})', 'Value': metrics['precision']})
        metrics_list.append({'Metric': f'Recall ({label})', 'Value': metrics['recall']})
        metrics_list.append({'Metric': f'F1-Score ({label})', 'Value': metrics['f1-score']})

# Save the DataFrame to a CSV file
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv('Models/random_forest_metrics.csv', index=False)

# Calculate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Create a DataFrame for ROC curve data
roc_data_df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'Thresholds': thresholds
})



# Save ROC curve data to CSV
roc_data_df.to_csv('Models/random_forest_roc_data.csv', index=False)

# Predict the labels for the unlabeled dataset
rf_y_unlabeled_pred = best_model.predict(X_unlabeled_scaled)

# Count the number of 0s and 1s in the predictions
num_zeros = np.sum(rf_y_unlabeled_pred == 0)
num_ones = np.sum(rf_y_unlabeled_pred == 1)

print(f"Number of 0s (normal): {num_zeros}")
print(f"Number of 1s (attack): {num_ones}")

unlabeled['prediction'] = rf_y_unlabeled_pred
platform_prediction_counts = unlabeled.groupby(['platform', 'prediction']).size().unstack(fill_value=0)
platform_prediction_counts.to_csv('Models/random_forest_unlabeled_platform_prediction_counts.csv')

unlabeled_predictions['Random_Forest'] = rf_y_unlabeled_pred

# Feature Importance
feature_importance = best_model.feature_importances_
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

[I 2025-01-30 15:24:41,341] A new study created in memory with name: no-name-f3a3f1c4-7095-4c52-9ddf-ffc059c48d01
[I 2025-01-30 15:25:06,782] Trial 0 finished with value: 0.9276573743251735 and parameters: {'n_estimators': 24, 'max_depth': 20, 'min_samples_split': 9}. Best is trial 0 with value: 0.9276573743251735.
[I 2025-01-30 15:25:19,919] Trial 1 finished with value: 0.0026646951909938834 and parameters: {'n_estimators': 157, 'max_depth': 1, 'min_samples_split': 3}. Best is trial 0 with value: 0.9276573743251735.
[I 2025-01-30 15:25:21,913] Trial 2 finished with value: 0.0026646951909938834 and parameters: {'n_estimators': 13, 'max_depth': 1, 'min_samples_split': 8}. Best is trial 0 with value: 0.9276573743251735.
[I 2025-01-30 15:25:42,679] Trial 3 finished with value: 0.8598051689758923 and parameters: {'n_estimators': 50, 'max_depth': 11, 'min_samples_split': 2}. Best is trial 0 with value: 0.9276573743251735.
[I 2025-01-30 15:26:40,053] Trial 4 finished with value: 0.9298755720

Trained and saved new Random Forest model.
Number of 0s (normal): 160810
Number of 1s (attack): 69946
                      Feature  Importance
6            flashloan_in_usd    0.298553
1           to_address_profit    0.272717
0         from_address_profit    0.208211
2       highest_profit_in_usd    0.092140
4                 path_length    0.089531
5             num_swap_events    0.030892
3  highest_price_change_ratio    0.007955


# K-Nearest Neighbors

In [4]:
# Define the objective function for Optuna
def knn_objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 50)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    
    # Use StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=skf, n_jobs=-1, scoring='f1').mean()
    return score

# Check if the model already exists
if os.path.exists(f'Models/{threshold}/knn_model_{threshold}.pkl'):
    # Load the existing model
    best_model = joblib.load(f'Models/{threshold}/knn_model_{threshold}.pkl')
    print("Loaded existing K-Nearest Neighbors model.")
    best_params = best_model.get_params()
    for param, value in best_params.items():
        print(f"{param}: {value}")
else: 
    # Optimize with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(knn_objective, n_trials=50)

    # Best parameters
    best_params = study.best_params
    best_model = KNeighborsClassifier(**best_params)
    # Save the model
    joblib.dump(best_model, f'Models/{threshold}/knn_model_{threshold}.pkl')
    print("Trained and saved new K-Nearest Neighbors model.")

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate confusion matrix and AUC-ROC score
conf_matrix = confusion_matrix(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

# Calculate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)

# Calculate precision, recall, and thresholds
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Create a DataFrame to store Precision-Recall data
pr_data_df = pd.DataFrame({
    'Recall': recall,
    'Precision': precision
})

# Save the Precision-Recall data to a CSV file
pr_data_df.to_csv('Models/knn_precision_recall_data.csv', index=False)

# Prepare confusion matrix data
tn, fp, fn, tp = conf_matrix.ravel()

# Create a DataFrame to store the metrics
metrics_list = [
    {'Metric': 'True Negatives', 'Value': tn},
    {'Metric': 'False Positives', 'Value': fp},
    {'Metric': 'False Negatives', 'Value': fn},
    {'Metric': 'True Positives', 'Value': tp},
    {'Metric': 'AUC-ROC', 'Value': auc_roc}
]

# Add classification report data to the metrics DataFrame
for label, metrics in class_report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        metrics_list.append({'Metric': f'Precision ({label})', 'Value': metrics['precision']})
        metrics_list.append({'Metric': f'Recall ({label})', 'Value': metrics['recall']})
        metrics_list.append({'Metric': f'F1-Score ({label})', 'Value': metrics['f1-score']})

# Save the DataFrame to a CSV file
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv('Models/knn_metrics.csv', index=False)

# Calculate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Create a DataFrame for ROC curve data
roc_data_df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'Thresholds': thresholds
})

# Save ROC curve data to CSV
roc_data_df.to_csv('Models/knn_roc_data.csv', index=False)

# Predict the labels for the unlabeled dataset
knn_y_unlabeled_pred = best_model.predict(X_unlabeled_scaled)

# Count the number of 0s and 1s in the predictions
num_zeros = np.sum(knn_y_unlabeled_pred == 0)
num_ones = np.sum(knn_y_unlabeled_pred == 1)

print(f"Number of 0s (normal): {num_zeros}")
print(f"Number of 1s (attack): {num_ones}")

unlabeled['prediction'] = knn_y_unlabeled_pred
platform_prediction_counts = unlabeled.groupby(['platform', 'prediction']).size().unstack(fill_value=0)
platform_prediction_counts.to_csv('Models/knn_unlabeled_platform_prediction_counts.csv')

unlabeled_predictions['KNN'] = knn_y_unlabeled_pred

[I 2025-01-30 16:08:47,828] A new study created in memory with name: no-name-0c72b275-3a7a-4b15-840c-efca0f4bb1fa
[I 2025-01-30 16:09:02,000] Trial 0 finished with value: 0.8287656597742922 and parameters: {'n_neighbors': 47, 'weights': 'distance'}. Best is trial 0 with value: 0.8287656597742922.
[I 2025-01-30 16:09:10,728] Trial 1 finished with value: 0.8409137067250654 and parameters: {'n_neighbors': 8, 'weights': 'uniform'}. Best is trial 1 with value: 0.8409137067250654.
[I 2025-01-30 16:09:19,859] Trial 2 finished with value: 0.8446430270465457 and parameters: {'n_neighbors': 6, 'weights': 'uniform'}. Best is trial 2 with value: 0.8446430270465457.
[I 2025-01-30 16:09:31,633] Trial 3 finished with value: 0.833627417039493 and parameters: {'n_neighbors': 37, 'weights': 'distance'}. Best is trial 2 with value: 0.8446430270465457.
[I 2025-01-30 16:09:43,051] Trial 4 finished with value: 0.8114221707038449 and parameters: {'n_neighbors': 23, 'weights': 'uniform'}. Best is trial 2 with

Trained and saved new K-Nearest Neighbors model.
Number of 0s (normal): 195024
Number of 1s (attack): 35732


# Decision Tree Classifier

In [5]:
# Define the objective function for Optuna
def dt_objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, random_state=42)
    
    # Use StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=skf, n_jobs=-1, scoring='f1').mean()
    return score

# Check if the model already exists
if os.path.exists(f'Models/{threshold}/decision_tree_model_{threshold}.pkl'):
    # Load the existing model
    best_model = joblib.load(f'Models/{threshold}/decision_tree_model_{threshold}.pkl')
    print("Loaded existing Decision Tree model.")
    best_params = best_model.get_params()
    for param, value in best_params.items():
        print(f"{param}: {value}")
else: 
    # Optimize with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(dt_objective, n_trials=50)

    # Best parameters
    best_params = study.best_params
    best_model = DecisionTreeClassifier(**best_params, random_state=42)
    # Save the model
    joblib.dump(best_model, f'Models/{threshold}/decision_tree_model_{threshold}.pkl')
    print("Trained and saved new Decision Tree model.")

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate confusion matrix and AUC-ROC score
conf_matrix = confusion_matrix(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

# Calculate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)

# Calculate precision, recall, and thresholds
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Create a DataFrame to store Precision-Recall data
pr_data_df = pd.DataFrame({
    'Recall': recall,
    'Precision': precision
})

# Save the Precision-Recall data to a CSV file
pr_data_df.to_csv('Models/decision_tree_precision_recall_data.csv', index=False)

# Prepare confusion matrix data
tn, fp, fn, tp = conf_matrix.ravel()

# Create a DataFrame to store the metrics
metrics_list = [
    {'Metric': 'True Negatives', 'Value': tn},
    {'Metric': 'False Positives', 'Value': fp},
    {'Metric': 'False Negatives', 'Value': fn},
    {'Metric': 'True Positives', 'Value': tp},
    {'Metric': 'AUC-ROC', 'Value': auc_roc}
]

# Add classification report data to the metrics DataFrame
for label, metrics in class_report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        metrics_list.append({'Metric': f'Precision ({label})', 'Value': metrics['precision']})
        metrics_list.append({'Metric': f'Recall ({label})', 'Value': metrics['recall']})
        metrics_list.append({'Metric': f'F1-Score ({label})', 'Value': metrics['f1-score']})

# Save the DataFrame to a CSV file
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv('Models/decision_tree_metrics.csv', index=False)

# Calculate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Create a DataFrame for ROC curve data
roc_data_df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'Thresholds': thresholds
})

# Save ROC curve data to CSV
roc_data_df.to_csv('Models/decision_tree_roc_data.csv', index=False)

# Predict the labels for the unlabeled dataset
dt_y_unlabeled_pred = best_model.predict(X_unlabeled_scaled)

# Count the number of 0s and 1s in the predictions
num_zeros = np.sum(dt_y_unlabeled_pred == 0)
num_ones = np.sum(dt_y_unlabeled_pred == 1)

print(f"Number of 0s (normal): {num_zeros}")
print(f"Number of 1s (attack): {num_ones}")

unlabeled['prediction'] = dt_y_unlabeled_pred
platform_prediction_counts = unlabeled.groupby(['platform', 'prediction']).size().unstack(fill_value=0)
platform_prediction_counts.to_csv('Models/decision_tree_unlabeled_platform_prediction_counts.csv')

unlabeled_predictions['Decision_Tree'] = dt_y_unlabeled_pred

# Feature Importance
feature_importance = best_model.feature_importances_
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

[I 2025-01-30 16:17:26,700] A new study created in memory with name: no-name-3bb93014-5eb9-4c96-914e-27872dfa8ac1
[I 2025-01-30 16:17:29,834] Trial 0 finished with value: 0.9160775088001021 and parameters: {'max_depth': 19, 'min_samples_split': 6}. Best is trial 0 with value: 0.9160775088001021.
[I 2025-01-30 16:17:32,579] Trial 1 finished with value: 0.8993064231614241 and parameters: {'max_depth': 15, 'min_samples_split': 2}. Best is trial 0 with value: 0.9160775088001021.
[I 2025-01-30 16:17:35,180] Trial 2 finished with value: 0.8986284026167322 and parameters: {'max_depth': 15, 'min_samples_split': 6}. Best is trial 0 with value: 0.9160775088001021.
[I 2025-01-30 16:17:37,692] Trial 3 finished with value: 0.8877140553048084 and parameters: {'max_depth': 14, 'min_samples_split': 9}. Best is trial 0 with value: 0.9160775088001021.
[I 2025-01-30 16:17:39,713] Trial 4 finished with value: 0.8547622686134483 and parameters: {'max_depth': 11, 'min_samples_split': 7}. Best is trial 0 wit

Trained and saved new Decision Tree model.
Number of 0s (normal): 156813
Number of 1s (attack): 73943
                      Feature  Importance
6            flashloan_in_usd    0.388594
1           to_address_profit    0.298802
2       highest_profit_in_usd    0.115471
4                 path_length    0.092235
0         from_address_profit    0.078233
5             num_swap_events    0.023285
3  highest_price_change_ratio    0.003380


# Gradient Boosting Classifier

In [6]:
# Define the objective function for Optuna
def gb_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    
    model = GradientBoostingClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    
    # Use StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=skf, n_jobs=-1, scoring='f1').mean()
    return score

# Check if the model already exists
if os.path.exists(f'Models/{threshold}/gradient_boosting_model_{threshold}.pkl'):
    # Load the existing model
    best_model = joblib.load(f'Models/{threshold}/gradient_boosting_model_{threshold}.pkl')
    print("Loaded existing Gradient Boosting model.")
    best_params = best_model.get_params()
    for param, value in best_params.items():
        print(f"{param}: {value}")
else: 
    # Optimize with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(gb_objective, n_trials=50)

    # Best parameters
    best_params = study.best_params
    best_model = GradientBoostingClassifier(**best_params, random_state=42)
    # Save the model
    joblib.dump(best_model, f'Models/{threshold}/gradient_boosting_model_{threshold}.pkl')
    print("Trained and saved new Gradient Boosting model.")

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate confusion matrix and AUC-ROC score
conf_matrix = confusion_matrix(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

# Calculate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)

# Calculate precision, recall, and thresholds
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Create a DataFrame to store Precision-Recall data
pr_data_df = pd.DataFrame({
    'Recall': recall,
    'Precision': precision
})

# Save the Precision-Recall data to a CSV file
pr_data_df.to_csv('Models/gradient_boosting_precision_recall_data.csv', index=False)

# Prepare confusion matrix data
tn, fp, fn, tp = conf_matrix.ravel()

# Create a DataFrame to store the metrics
metrics_list = [
    {'Metric': 'True Negatives', 'Value': tn},
    {'Metric': 'False Positives', 'Value': fp},
    {'Metric': 'False Negatives', 'Value': fn},
    {'Metric': 'True Positives', 'Value': tp},
    {'Metric': 'AUC-ROC', 'Value': auc_roc}
]

# Add classification report data to the metrics DataFrame
for label, metrics in class_report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        metrics_list.append({'Metric': f'Precision ({label})', 'Value': metrics['precision']})
        metrics_list.append({'Metric': f'Recall ({label})', 'Value': metrics['recall']})
        metrics_list.append({'Metric': f'F1-Score ({label})', 'Value': metrics['f1-score']})

# Save the DataFrame to a CSV file
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv('Models/gradient_boosting_metrics.csv', index=False)

# Calculate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Create a DataFrame for ROC curve data
roc_data_df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'Thresholds': thresholds
})

# Save ROC curve data to CSV
roc_data_df.to_csv('Models/gradient_boosting_roc_data.csv', index=False)

# Predict the labels for the unlabeled dataset
gb_y_unlabeled_pred = best_model.predict(X_unlabeled_scaled)

# Count the number of 0s and 1s in the predictions
num_zeros = np.sum(gb_y_unlabeled_pred == 0)
num_ones = np.sum(gb_y_unlabeled_pred == 1)

print(f"Number of 0s (normal): {num_zeros}")
print(f"Number of 1s (attack): {num_ones}")

unlabeled['prediction'] = gb_y_unlabeled_pred
platform_prediction_counts = unlabeled.groupby(['platform', 'prediction']).size().unstack(fill_value=0)
platform_prediction_counts.to_csv('Models/gradient_boosting_unlabeled_platform_prediction_counts.csv')

unlabeled_predictions['Gradient_Boosting'] = gb_y_unlabeled_pred

# Feature Importance
feature_importance = best_model.feature_importances_
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

[I 2025-01-30 16:19:43,948] A new study created in memory with name: no-name-f5b91147-eadb-41b2-9193-9426eee318d3
[I 2025-01-30 16:22:33,249] Trial 0 finished with value: 0.9030958631883393 and parameters: {'n_estimators': 153, 'max_depth': 7}. Best is trial 0 with value: 0.9030958631883393.
[I 2025-01-30 16:24:16,607] Trial 1 finished with value: 0.9045524783945753 and parameters: {'n_estimators': 72, 'max_depth': 9}. Best is trial 1 with value: 0.9045524783945753.
[I 2025-01-30 16:24:46,772] Trial 2 finished with value: 0.8739906562102935 and parameters: {'n_estimators': 20, 'max_depth': 9}. Best is trial 1 with value: 0.9045524783945753.
[I 2025-01-30 16:26:27,383] Trial 3 finished with value: 0.8495102792918946 and parameters: {'n_estimators': 194, 'max_depth': 3}. Best is trial 1 with value: 0.9045524783945753.
[I 2025-01-30 16:29:37,342] Trial 4 finished with value: 0.9049423680413599 and parameters: {'n_estimators': 173, 'max_depth': 7}. Best is trial 4 with value: 0.90494236804

Trained and saved new Gradient Boosting model.
Number of 0s (normal): 151976
Number of 1s (attack): 78780
                      Feature  Importance
6            flashloan_in_usd    0.373155
1           to_address_profit    0.255845
2       highest_profit_in_usd    0.126713
0         from_address_profit    0.120482
4                 path_length    0.089563
5             num_swap_events    0.028792
3  highest_price_change_ratio    0.005450


# AdaBoost Classifier

In [7]:
# Define the objective function for Optuna
def ab_objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 400)
    model = AdaBoostClassifier(n_estimators=n_estimators, algorithm='SAMME', random_state=42)
    
    # Use StratifiedKFold for stratified cross-validation
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=skf, n_jobs=-1, scoring='f1').mean()
    return score

# Check if the model already exists
if os.path.exists(f'Models/{threshold}/adaboost_model_{threshold}.pkl'):
    # Load the existing model
    best_model = joblib.load(f'Models/{threshold}/adaboost_model_{threshold}.pkl')
    print("Loaded existing AdaBoost model.")
    best_params = best_model.get_params()
    for param, value in best_params.items():
        print(f"{param}: {value}")
else: 
    # Optimize with Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(ab_objective, n_trials=50)

    # Best parameters
    best_params = study.best_params
    best_model = AdaBoostClassifier(**best_params, algorithm='SAMME', random_state=42)
    # Save the model
    joblib.dump(best_model, f'Models/{threshold}/adaboost_model_{threshold}.pkl')
    print("Trained and saved new AdaBoost model.")

# Fit and evaluate
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class

# Calculate confusion matrix and AUC-ROC score
conf_matrix = confusion_matrix(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)

# Calculate classification report
class_report = classification_report(y_test, y_pred, output_dict=True)

# Calculate precision, recall, and thresholds
precision, recall, _ = precision_recall_curve(y_test, y_proba)

# Create a DataFrame to store Precision-Recall data
pr_data_df = pd.DataFrame({
    'Recall': recall,
    'Precision': precision
})

# Save the Precision-Recall data to a CSV file
pr_data_df.to_csv('Models/adaboost_precision_recall_data.csv', index=False)

# Prepare confusion matrix data
tn, fp, fn, tp = conf_matrix.ravel()

# Create a DataFrame to store the metrics
metrics_list = [
    {'Metric': 'True Negatives', 'Value': tn},
    {'Metric': 'False Positives', 'Value': fp},
    {'Metric': 'False Negatives', 'Value': fn},
    {'Metric': 'True Positives', 'Value': tp},
    {'Metric': 'AUC-ROC', 'Value': auc_roc}
]

# Add classification report data to the metrics DataFrame
for label, metrics in class_report.items():
    if label not in ['accuracy', 'macro avg', 'weighted avg']:
        metrics_list.append({'Metric': f'Precision ({label})', 'Value': metrics['precision']})
        metrics_list.append({'Metric': f'Recall ({label})', 'Value': metrics['recall']})
        metrics_list.append({'Metric': f'F1-Score ({label})', 'Value': metrics['f1-score']})

# Save the DataFrame to a CSV file
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv('Models/adaboost_metrics.csv', index=False)

# Calculate ROC curve data
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Create a DataFrame for ROC curve data
roc_data_df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr,
    'Thresholds': thresholds
})

# Save ROC curve data to CSV
roc_data_df.to_csv('Models/adaboost_roc_data.csv', index=False)

# Predict the labels for the unlabeled dataset
ada_y_unlabeled_pred = best_model.predict(X_unlabeled_scaled)

# Count the number of 0s and 1s in the predictions
num_zeros = np.sum(ada_y_unlabeled_pred == 0)
num_ones = np.sum(ada_y_unlabeled_pred == 1)

print(f"Number of 0s (normal): {num_zeros}")
print(f"Number of 1s (attack): {num_ones}")

unlabeled['prediction'] = ada_y_unlabeled_pred
platform_prediction_counts = unlabeled.groupby(['platform', 'prediction']).size().unstack(fill_value=0)
platform_prediction_counts.to_csv('Models/adaboost_unlabeled_platform_prediction_counts.csv')

unlabeled_predictions['AdaBoost'] = ada_y_unlabeled_pred

# Feature Importance
feature_importance = best_model.feature_importances_
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

[I 2025-01-30 21:13:13,623] A new study created in memory with name: no-name-5b20e78e-0f6d-4335-baf9-0f2c653e59bc
[I 2025-01-30 21:14:33,519] Trial 0 finished with value: 0.7155092416416393 and parameters: {'n_estimators': 313}. Best is trial 0 with value: 0.7155092416416393.
[I 2025-01-30 21:15:35,746] Trial 1 finished with value: 0.7139147083452613 and parameters: {'n_estimators': 291}. Best is trial 0 with value: 0.7155092416416393.
[I 2025-01-30 21:15:43,495] Trial 2 finished with value: 0.6146255902904904 and parameters: {'n_estimators': 30}. Best is trial 0 with value: 0.7155092416416393.
[I 2025-01-30 21:16:16,534] Trial 3 finished with value: 0.6742628141815106 and parameters: {'n_estimators': 137}. Best is trial 0 with value: 0.7155092416416393.
[I 2025-01-30 21:17:43,272] Trial 4 finished with value: 0.7212756111309053 and parameters: {'n_estimators': 387}. Best is trial 4 with value: 0.7212756111309053.
[I 2025-01-30 21:18:25,070] Trial 5 finished with value: 0.6872378830116

Trained and saved new AdaBoost model.
Number of 0s (normal): 188323
Number of 1s (attack): 42433
                      Feature  Importance
1           to_address_profit    0.410185
6            flashloan_in_usd    0.273694
0         from_address_profit    0.224647
4                 path_length    0.077582
5             num_swap_events    0.013892
2       highest_profit_in_usd    0.000000
3  highest_price_change_ratio    0.000000


In [8]:
unlabeled['Random_Forest'] = unlabeled_predictions['Random_Forest']
unlabeled['KNN'] = unlabeled_predictions['KNN']
unlabeled['Decision_Tree'] = unlabeled_predictions['Decision_Tree']
unlabeled['Gradient_Boosting'] = unlabeled_predictions['Gradient_Boosting']
unlabeled['AdaBoost'] = unlabeled_predictions['AdaBoost']
unlabeled_predictions = unlabeled_predictions.astype(bool)
unlabeled['All_Attack'] = unlabeled_predictions.sum(axis=1) == len(unlabeled_predictions.columns)
unlabeled.to_csv('Models/unlabeled_all_models.csv', index=False)
all_attack_indices = unlabeled[unlabeled['All_Attack']].index
all_attack_data = unlabeled.loc[all_attack_indices]
all_attack_data.to_csv('Models/unlabeled_all_attack_data.csv', index=False)